In [26]:
import os
import sys
import numpy as np
import matplotlib.image as image
from PIL import Image as im
import random

#### Filter the center point of each traffic-light from all traffic-light points

In [27]:
def minimize_locations(locations):
    rows, cols = locations[0], locations[1]
    x_coordinate, y_coordinate = [], []
    if len(rows) and len(cols):
        first_point = (rows[0], cols[0])
    for i in range(1, len(rows)):
        if rows[i - 1] + 1 < rows[i]:
            mid_row = (first_point[0] + rows[i - 1]) // 2
            mid_col = (first_point[1] + cols[i - 1]) // 2
            x_coordinate += [mid_col]
            y_coordinate += [mid_row]
            first_point = (rows[i], cols[i])     
    return x_coordinate, y_coordinate

#### return random non traffic-light points from image 

In [28]:
def get_random_coord(length, image):
    non_traffic_lights_locations=([],[])
    for _ in range(length):
        flag=False
        while not flag:
            flag = True
            row = random.randint(0,942)
            col = random.randint(0,1966)
            for i in range(81):
                for j in range(81):
                    if row+i < 1024 and col+j < 2048 and image[row+i, col+j] == 19:
                        flag = False
                        break
            if flag:
                non_traffic_lights_locations[0].append(row)
                non_traffic_lights_locations[1].append(col)
    return non_traffic_lights_locations

#### Return dict {key=name of image: value=(traffic-light points, non traffic-light points)}

In [29]:
def load_data(folder_path, data_kind):
    folder_path = folder_path + 'gtFine/' + data_kind
    traffic_lights_locations_dict={}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('labelIds.png'):
                image_path = root+'/'+file
                image = np.asarray(im.open(image_path))
                traffic_lights_locations = np.where(image==19)
                traffic_lights_locations = minimize_locations(traffic_lights_locations)
                non_traffic_lights_locations = get_random_coord(len(traffic_lights_locations[0]), image)
                traffic_lights_locations_dict[file[:file.find('_gtFine_labelIds.png')]] = (traffic_lights_locations[0], traffic_lights_locations[1], non_traffic_lights_locations[0], non_traffic_lights_locations[1])
    return traffic_lights_locations_dict

#### Return images as np arrays and their classification.

In [39]:
def classify_and_cut(folder_path, data_kind, tfl_locations):
    folder_path = folder_path + 'data/' + 'leftImg8bit/' + data_kind
    images, classify = [], []
    for root, dirs, files in os.walk(folder_path):
        for file in files:            
            if file.endswith('_leftImg8bit.png'):
                file_path = root + '/' + file
                image_name = file[:file.find('_leftImg8bit.png')]
                x_tfl , y_tfl, x_random, y_random = tfl_locations[image_name][0], tfl_locations[image_name][1], tfl_locations[image_name][2], tfl_locations[image_name][3]
                image = im.open(file_path)
                for i in range(len(x_tfl)):
                    x, y = x_tfl[i], y_tfl[i]
                    w, h = image.size[0] - 1, image.size[1] - 1
                    left, top, right, bottom = max(0, x - 40), max(y - 40, 0), min(w, x + 41), min(y + 41, h)
                    if x - 40 < 0:
                        left, right = 0, 81
                    if y - 40 < 0:
                        top, bottom = 0, 81
                    if x + 41 > w:
                        left, right = w - 81, w
                    if y + 41 > h:
                        top, bottom = h - 81, h
                    crop_im = image.crop((left, top, right, bottom))
                    images.append(np.asarray(crop_im, dtype=np.uint8))
                    classify.append(1)
                for i in range(len(x_random)):
                    x, y = x_random[i], y_random[i]
                    left, top, right, bottom= y, x, y + 81,x + 81
                    crop_im = image.crop((left, top, right, bottom))
                    images.append(np.asarray(crop_im, dtype=np.uint8))
                    classify.append(0)
    return images, classify

#### Save data to binary files

In [40]:
def save_data(images, classify, file_path):
    for i in range(len(images)):
        data = file_path+'data.bin'
        label = file_path+'labels.bin'
        image = np.asarray(images[i], dtype=np.uint8)
        with open(data, 'ab') as data_bin:
            data_bin.write(image.tobytes())
        with open(label, 'ab') as labels_bin:
            labels_bin.write(classify[i].to_bytes(1, 'little'))

#### The main - building dataset

In [41]:
def dataset_building(data_kind):
    path = '.../'
    tfl_locations_dict = load_data(path, data_kind)
    images, classify = classify_and_cut(path, data_kind, tfl_locations_dict)
    file_path = './Data_dir/' + data_kind + '/'
    save_data(images, classify, file_path)

In [42]:
dataset_building('train')

.../gtFine/train


In [22]:
dataset_building('val')

.../gtFine/val
